In [1]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
    
    

time: 0 ns (started: 2022-08-03 12:15:15 +05:30)


# Approach for Implementation - CPA 10th Round Key Extraction

Since it is proposed to extract the 10th round key using power traces provided, the approach is to perform an analysis of the back propagation of the cipher, till the S-BOX. 3 input files are provided to us: Trace Files - Contains 1000 traces, each with 12000 sample points for different inputs, textin_array - Contains input files (plaintext) with 1000 inputs (rows) of 128 bit (16 byte) each, textout_array - Contains the cipher text received after the AES is performed on the input files. 

The textout_array file will be used for the back propagation. Keyguesses will be made for each subkey. Each keyguess byte will be ex-ored with the corresponding position byte of the cipher text, for all output rows. In AES, this step is preceeded by the Shift Rows operation in the 10th round (there is no Mixed Column operation in the 10th round). HOwever, since the shift row does not change the value but only its position, this may be ignored in the implementation for a test project. Since the propagation of key extraction is flowing backwards, the inverse shift row function should be performed, if it is planned. 

Similarly, the SBox operation preceeds the Shift Row function.Therefore, in back propagation, inverse S-Box has to be performed to arrive at the correct stage where CPA can be done with the power traces. SInce the S-Box is the non-linear function of the AES implementation, CPA is performed with tangible results in this phase. 

From the power traces (trace_array), we notice that the 10th round does not have the mixed column in implementation. Therefore, as observed from the traces, the sample points 9550 to 9700 depict the s-box operations against which we would like to correlate. 

In [2]:
def sbox(inp):
    s =  [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
            0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
            0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
            0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
            0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
            0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
            0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
            0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
            0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
            0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
            0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
            0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
            0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
            0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
            0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
            0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
            0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
            0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
            0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
            0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
            0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
            0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
            0x54, 0xbb, 0x16]
    return s[inp]


def invsbox(inp):
    s = [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3,
            0x9e, 0x81, 0xf3, 0xd7, 0xfb , 0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f,
            0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb , 0x54,
            0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b,
            0x42, 0xfa, 0xc3, 0x4e , 0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24,
            0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25 , 0x72, 0xf8,
            0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d,
            0x65, 0xb6, 0x92 , 0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda,
            0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84 , 0x90, 0xd8, 0xab,
            0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3,
            0x45, 0x06 , 0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1,
            0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b , 0x3a, 0x91, 0x11, 0x41,
            0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6,
            0x73 , 0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9,
            0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e , 0x47, 0xf1, 0x1a, 0x71, 0x1d,
            0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b ,
            0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0,
            0xfe, 0x78, 0xcd, 0x5a, 0xf4 , 0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07,
            0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f , 0x60,
            0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f,
            0x93, 0xc9, 0x9c, 0xef , 0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5,
            0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61 , 0x17, 0x2b,
            0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55,
            0x21, 0x0c, 0x7d]
    return s[inp]


def intermediate(pt, keyguess):
    return invsbox(pt ^ keyguess)

HW = [bin(n).count("1") for n in range(0, 256)]

time: 16 ms (started: 2022-08-03 12:15:17 +05:30)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange
from scipy.stats import linregress
import seaborn as sns
import time
import rich as r
import pandas as pd
from IPython.display import clear_output # type: ignore
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer

# load the .npy files
#known_keys = np.load('cw_traces/known_keys.npy')
textout_array = np.load('AssignmentFiles/textout_array.npy') # Contains the Cipher Text which will be used for key extraction
textin_array = np.load('AssignmentFiles/textin_array.npy') # Plain text... Not used here
trace_array = np.load('AssignmentFiles/trace_array.npy') # Power Trace FIles - We are looking at 9550 to 9700 sample points

print(textout_array.shape)
print(textin_array.shape)
print(trace_array.shape)

(1000, 16)
(1000, 16)
(1000, 12000)
time: 203 ms (started: 2022-08-03 12:17:51 +05:30)


In [5]:
output_notebook()
p = figure(plot_width=4000, plot_height=400)
p.line(range(len(trace_array[0])),trace_array[0])

show(p)

Loading BokehJS ...

time: 344 ms (started: 2022-08-03 12:18:06 +05:30)


# CPA

In [6]:
fmt = "{:02X}<br>{:.3f}"
def format_stat(stat):
    return str(fmt.format(stat[0], stat[1]))

def color_corr_key(row):
    # print(len(row))
    ret = [""] * len(row)
    for i, bnum in enumerate(row):
        if i == 0:
            ret[i] = "color: green"
        else:
            ret[i] = "color: red"
    return ret

time: 15 ms (started: 2022-08-03 18:37:10 +05:30)


In [7]:
key_guess = []
numtraces = trace_array.shape[0]

numpoints = trace_array.shape[1]
start_point = 9550 # Start of S-Box implementation for 10th Round AES
end_point = 9700 # End of S-Box implementation for 10th Round AES

crvs = np.zeros((16, 256, numpoints))
HW_matrix = np.zeros((numtraces, 256), dtype=np.uint8)

printable = []    # for printing the top guessed keys


for subkey in tnrange(0, 16, desc="Attacking Subkey"):# For each subkey in the 128 bit AES key of the 10th round
    temp = []    # it holds the [(key_guess1, correlation_value),(key_guess2, correlation_value),..]
    for kguess in tnrange(0, 256, desc="Generating Hamming Weights"):
        # Generate the hamming weight for each possible keybyte
        for trace_no in range(numtraces):
            iv = intermediate(textout_array[trace_no][subkey], kguess)
            HW_matrix[trace_no, kguess] = HW[iv]

        # correlate the trace with the hamming weights
        for point in range(start_point, end_point):
            hw = HW_matrix[:, kguess]
            trc = trace_array[:, point]
            crvs[subkey, kguess, point] = np.abs(linregress(hw, trc).slope)

        temp.append((kguess, np.max(crvs[subkey, kguess])))

    temp.sort(key = lambda x: -x[1])  # sort temp by dom value
    printable.append(temp)  # add the data in list
    df = pd.DataFrame(printable).transpose()

    key_guess.append(crvs[subkey].max(axis=1).argmax())

    clear_output(wait=True)  # clear the previous output
    display(df.head().style.format(format_stat).apply(color_corr_key, axis=0))  # display the current status


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,970.004,B30.003,170.003,5E0.004,970.002,3C0.004,A20.003,540.002,280.002,E30.002,4E0.003,C60.002,BE0.002,330.003,7C0.003,B70.002
1,E50.002,2C0.002,650.001,8F0.001,1B0.002,A00.002,530.001,F10.001,B50.002,830.002,6D0.002,060.001,DD0.002,530.002,C50.001,850.001
2,F50.002,2E0.002,D70.001,F50.001,E40.002,5F0.002,790.001,080.001,5A0.002,7C0.002,C20.002,2B0.001,DE0.002,120.002,AD0.001,040.001
3,0A0.002,D00.002,E60.001,360.001,E20.001,B00.002,CA0.001,C40.001,B00.002,7D0.002,D30.002,620.001,260.002,520.002,870.001,EF0.001
4,1F0.001,3F0.002,9A0.001,810.001,FD0.001,6F0.002,730.001,F60.001,420.002,100.002,D20.002,C70.001,9D0.002,590.002,9E0.001,3A0.001


time: 6min 20s (started: 2022-08-03 18:37:12 +05:30)


In [12]:
print(key_guess)

[151, 179, 23, 94, 151, 60, 162, 84, 40, 227, 78, 198, 190, 51, 124, 183]
time: 0 ns (started: 2022-08-03 19:06:39 +05:30)


In [13]:
%unload_ext autotime